# Data loading

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd    
import matplotlib.pyplot as plt
from tabulate import tabulate
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
credit = pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv')  
application = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv') 

# Definition of "Bad" client

Detailed explanation could be seen [here](https://www.listendata.com/2019/09/credit-risk-vintage-analysis.html). 


_This part was adapted from a notebook of @Xiao Song_

In [ ]:
grouped = credit.groupby('ID')
### convert credit data to wide format which every ID is a row
pivot_tb = credit.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')
pivot_tb['open_month'] = grouped['MONTHS_BALANCE'].min() # smallest value of MONTHS_BALANCE, is the month when loan was granted
pivot_tb['end_month'] = grouped['MONTHS_BALANCE'].max() # biggest value of MONTHS_BALANCE, might be observe over or canceling account
pivot_tb['ID'] = pivot_tb.index
pivot_tb = pivot_tb[['ID', 'open_month', 'end_month']]
pivot_tb['window'] = pivot_tb['end_month'] - pivot_tb['open_month'] # calculate observe window
pivot_tb.reset_index(drop = True, inplace = True)
credit = pd.merge(credit, pivot_tb, on = 'ID', how = 'left') # join calculated information
credit0 = credit.copy()
credit = credit[credit['window'] > 20] # delete users whose observe window less than 20
credit['status'] = np.where((credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 1, 0) # analyze > 60 days past due 
credit['status'] = credit['status'].astype(np.int8) # 1: overdue 0: not
credit['month_on_book'] = credit['MONTHS_BALANCE'] - credit['open_month'] # calculate month on book: how many months after opening account
credit.sort_values(by = ['ID','month_on_book'], inplace = True)

##### denominator
denominator = pivot_tb.groupby(['open_month']).agg({'ID': ['count']}) # count how many users in every month the account was opened
denominator.reset_index(inplace = True)
denominator.columns = ['open_month','sta_sum']

##### ventage table
vintage = credit.groupby(['open_month','month_on_book']).agg({'ID': ['count']}) 
vintage.reset_index(inplace = True)
vintage.columns = ['open_month','month_on_book','sta_sum'] 
vintage['due_count'] = np.nan
vintage = vintage[['open_month','month_on_book','due_count']] # delete aggerate column
vintage = pd.merge(vintage, denominator, on = ['open_month'], how = 'left') # join sta_sum colun to vintage table

In [ ]:
larger_window = abs(vintage['open_month'].min())
for j in range(-larger_window,1): # outer loop: month in which account was opened
    ls = []
    for i in range(0,larger_window+1): # inner loop time after the credit card was granted
        due = list(credit[(credit['status'] == 1) & (credit['month_on_book'] == i) & (credit['open_month'] == j)]['ID']) # get ID which satisfy the condition
        ls.extend(due) # As time goes, add bad customers
        vintage.loc[(vintage['month_on_book'] == i) & (vintage['open_month'] == j), 'due_count'] = len(set(ls)) # calculate non-duplicate ID numbers using set()
        
vintage['sta_rate']  = vintage['due_count'] / vintage['sta_sum'] # calculate cumulative % of bad customers        

In [ ]:
def calculate_observe(credit, command):
    '''calculate observe window
    '''
    larger_window = abs(credit['MONTHS_BALANCE'].min())
    id_sum = len(set(pivot_tb['ID']))
    credit['status'] = 0
    exec(command)
    #credit.loc[(credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1
    credit['month_on_book'] = credit['MONTHS_BALANCE'] - credit['open_month']
    minagg = credit[credit['status'] == 1].groupby('ID')['month_on_book'].min()
    minagg = pd.DataFrame(minagg)
    minagg['ID'] = minagg.index
    obslst = pd.DataFrame({'month_on_book':range(0,larger_window + 1), 'rate': None})
    lst = []
    for i in range(0,larger_window + 1):
        due = list(minagg[minagg['month_on_book']  == i]['ID'])
        lst.extend(due)
        obslst.loc[obslst['month_on_book'] == i, 'rate'] = len(set(lst)) / id_sum 
    return obslst['rate']

command = "credit.loc[(credit['STATUS'] == '0') | (credit['STATUS'] == '1') | (credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"   
morethan1 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '1') | (credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"   
morethan30 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan60 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan90 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 'status'] = 1"
morethan120 = calculate_observe(credit, command)
command = "credit.loc[(credit['STATUS'] == '5'), 'status'] = 1"
morethan150 = calculate_observe(credit, command)

In [ ]:
def calculate_rate(pivot_tb, command): 
    '''calculate bad customer rate
    '''
    credit0['status'] = None
    exec(command) # excuate input code
    sumagg = credit0.groupby('ID')['status'].agg(sum)
    pivot_tb = pd.merge(pivot_tb, sumagg, on = 'ID', how = 'left')
    pivot_tb.loc[pivot_tb['status'] > 1, 'status'] = 1
    rate = pivot_tb['status'].sum() / len(pivot_tb)
    return round(rate, 5)

command = "credit0.loc[(credit0['STATUS'] == '0') | (credit0['STATUS'] == '1') | (credit0['STATUS'] == '2') | (credit0['STATUS'] == '3' )| (credit0['STATUS'] == '4' )| (credit0['STATUS'] == '5'), 'status'] = 1"   
morethan1 = calculate_rate(pivot_tb, command)
command = "credit0.loc[(credit0['STATUS'] == '1') | (credit0['STATUS'] == '2') | (credit0['STATUS'] == '3' )| (credit0['STATUS'] == '4' )| (credit0['STATUS'] == '5'), 'status'] = 1"   
morethan30 = calculate_rate(pivot_tb, command)
command = "credit0.loc[(credit0['STATUS'] == '2') | (credit0['STATUS'] == '3' )| (credit0['STATUS'] == '4' )| (credit0['STATUS'] == '5'), 'status'] = 1"
morethan60 = calculate_rate(pivot_tb, command)
command = "credit0.loc[(credit0['STATUS'] == '3' )| (credit0['STATUS'] == '4' )| (credit0['STATUS'] == '5'), 'status'] = 1"
morethan90 = calculate_rate(pivot_tb, command)
command = "credit0.loc[(credit0['STATUS'] == '4' )| (credit0['STATUS'] == '5'), 'status'] = 1"
morethan120 = calculate_rate(pivot_tb, command)
command = "credit0.loc[(credit0['STATUS'] == '5'), 'status'] = 1"
morethan150 = calculate_rate(pivot_tb, command)

## Definition of targets

Possibilities : 
1. Past due more than X days
2. Past more than Y% of dues

In [ ]:
#"Bad" client are identified as client that past due more than 30 days
y = credit0[['ID','STATUS','status']]
y['status'] = 0 #0 is the label for a "good" client
exec("y.loc[(y['STATUS'] == '1') | (y['STATUS'] == '2') | (y['STATUS'] == '3' )| (y['STATUS'] == '4' )| (y['STATUS'] == '5'), 'status'] = 1") #1 is the label for a "Bad" client
y = y[['ID','status']].rename(columns={"ID": "ID", "status": "target"})
y

## Features engineering

In [ ]:
#Numerical features
application = application.replace(['N','Y'],[0,1]) #Converts Yes/No in 1/0
application = application.rename(columns={"CODE_GENDER":"F", "NAME_EDUCATION_TYPE":"EDUCATION"})
application = application.replace(['F','M'],[1,0]) #Converts Female/Male in 1/0
application = application.replace(['Academic degree', 'Higher education', 'Incomplete higher', 
                                   'Secondary / secondary special', 'Lower secondary'],
                                  [4,3,2,1,0]) #Converts education level into numerical features

In [ ]:
#Convert categorical variables
application['OCCUPATION_TYPE'] = application['OCCUPATION_TYPE'].apply(lambda x : 'Unknown' if pd.isnull(x) else x)
application = pd.get_dummies(application)

In [ ]:
#Normalize the dataset

# apply the maximum absolute scaling in Pandas using the .abs() and .max() methods
def normal_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # list to save the normal coef
    normal_coefs = []
    #We don't want to normalize the ID
    columns = list(df.columns)
    columns.remove("ID") 
    for column in columns:
        normal_coefs.append((df_scaled[column].mean(),df_scaled[column].std()))
        df_scaled[column] = (df_scaled[column]-normal_coefs[-1][0]) / normal_coefs[-1][1]
    return df_scaled, normal_coefs
    
# call the maximum_absolute_scaling function
application, normal_coefs = normal_scaling(application)

application = application.fillna(0)

# Deep Learning

In [ ]:
#Merge both datasets
train_valid_dataset = application
train_valid_dataset = train_valid_dataset.merge(y, on="ID", how="inner").drop(columns=["ID"])
#train_valid_dataset = train_valid_dataset[:10000] #Test only
train_valid_dataset = train_valid_dataset.to_numpy()

In [ ]:
#Split between train and test set

import torch
import torchvision.transforms as transforms

valid_ratio = 0.2  # Going to use 80%/20% split for train/valid
weight_tensor = torch.tensor([len(train_valid_dataset)/(len(train_valid_dataset)-train_valid_dataset[:,-1].sum()), 
                              len(train_valid_dataset)/train_valid_dataset[:,-1].sum()]).float() #weight_matrix

# Split it into training and validation sets
nb_train = int((1.0 - valid_ratio) * len(train_valid_dataset))
nb_valid =  int(valid_ratio * len(train_valid_dataset))
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(train_valid_dataset, [nb_train, nb_valid])

#Define device
use_gpu = torch.cuda.is_available()
if use_gpu:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
#Convert the dataset to a tensor

class DatasetTransformer(torch.utils.data.Dataset):

    def __init__(self, base_dataset, transform=transforms.Lambda(lambda x: x)):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, index):
        inpt, target = torch.from_numpy(self.base_dataset[index][:-1]), self.base_dataset[index][-1]
        return self.transform(inpt).float(), int(target)

    def __len__(self):
        return len(self.base_dataset)


train_dataset = DatasetTransformer(train_dataset)
valid_dataset = DatasetTransformer(valid_dataset)

In [ ]:
#Dataloader

num_threads = 4     # Loading the dataset is using Y CPU threads
batch_size  = 1024   # Using minibatches of X samples

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True,              # <-- this reshuffles the data at every epoch
                                          num_workers=num_threads)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False,
                                          num_workers=num_threads)


print("The train set contains {} samples, in {} batches".format(len(train_loader.dataset), len(train_loader)))
print("The validation set contains {} samples, in {} batches".format(len(valid_loader.dataset), len(valid_loader)))

In [ ]:
#Neural Network with fully connected layers

import torch.nn as nn

def linear_relu(dim_in, dim_out):
    return [nn.Linear(dim_in, dim_out),
            nn.ReLU(inplace=True)]

class FullyConnected(nn.Module):

    def __init__(self, input_size, num_classes):
        super(FullyConnected, self).__init__()
        self.classifier =  nn.Sequential(
            #nn.Dropout(0.2),
            *linear_relu(input_size, 32),
            #nn.Dropout(0.5), #Generally 0.2 for the input layer and 0.5 for the hidden layer
            *linear_relu(32, 32),
            #nn.Dropout(0.5),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size()[0], -1)
        y = self.classifier(x)
        return y


model = FullyConnected(48, 2)
model.to(device)

In [ ]:
def train(model, loader, f_loss, optimizer, device):
    """
    Train a model for one epoch, iterating over the loader
    using the f_loss to compute the loss and the optimizer
    to update the parameters of the model.

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        f_loss    -- The loss function, i.e. a loss Module
        optimizer -- A torch.optim.Optimzer object
        device    -- a torch.device class specifying the device
                     used for computation

    Returns :
    """

    # We enter train mode. This is useless for the linear model
    # but is important for layers such as dropout, batchnorm, ...
    model.train()

    for i, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Compute the forward pass through the network up to the loss
        outputs = model(inputs)
        loss = f_loss(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def test(model, loader, f_loss, device):
    """
    Test a model by iterating over the loader

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        f_loss    -- The loss function, i.e. a loss Module
        device    -- The device to use for computation 

    Returns :

        A tuple with the mean loss, mean accuracy and mean unbiaised accuracy

    """
    # We disable gradient computation which speeds up the computation
    # and reduces the memory usage
    with torch.no_grad():
        # We enter evaluation mode. This is useless for the linear model
        # but is important with layers such as dropout, batchnorm, ..
        model.eval()
        N = 0
        tot_loss, correct, unbiaised_acc = 0.0, 0.0, 0.0
        for i, (inputs, targets) in enumerate(loader):

            # We got a minibatch from the loader within inputs and targets

            # We need to copy the data on the GPU if we use one
            inputs, targets = inputs.to(device), targets.to(device)

            # Compute the forward pass, i.e. the scores for each input
            outputs = model(inputs)

            # We accumulate the exact number of processed samples
            N += inputs.shape[0]

            # We accumulate the loss considering
            # The multipliation by inputs.shape[0] is due to the fact
            # that our loss criterion is averaging over its samples
            tot_loss += inputs.shape[0] * f_loss(outputs, targets).item()

            # For the accuracy, we compute the labels for each input
            # Be carefull, the model is outputing scores and not the probabilities
            # But given the softmax is not altering the rank of its input scores
            # we can compute the label by argmaxing directly the scores
            predicted_targets = outputs.argmax(dim=1)
            correct += (predicted_targets == targets).sum().item()
            
            #Compute the unbiaised accuracy
            for value in predicted_targets.unique() :
                mask = (predicted_targets == targets) & (predicted_targets == value.item())
                unbiaised_acc += mask.sum().item()/(targets == value.item()).sum().item() * (inputs.shape[0]/len(predicted_targets.unique()))
            
        return tot_loss/N, correct/N, unbiaised_acc/N

In [ ]:
#To save the best model over epochs

class ModelCheckpoint:

    def __init__(self, filepath, model):
        self.min_loss = None
        self.filepath = filepath
        self.model = model

    def update(self, loss):
        if (self.min_loss is None) or (loss < self.min_loss):
            print("Saving a better model")
            torch.save(self.model.state_dict(), self.filepath)
            self.min_loss = loss
            
            
model_path = "best_model.pt"

In [ ]:
epochs = 50
optimizer = torch.optim.Adam(model.parameters())
f_loss = torch.nn.CrossEntropyLoss(weight=weight_tensor.to(device))
model_checkpoint = ModelCheckpoint(model_path, model)

for t in range(epochs):
    print("\nEpoch {}".format(t))
    train(model, train_loader, f_loss, optimizer, device)
    train_loss, train_acc, train_unb_acc = test(model, train_loader, f_loss, device)
    print(" Train : Loss : {:.4f}, Acc : {:.4f}, Unb.Acc. : {:.4f}".format(train_loss, train_acc, train_unb_acc))

    val_loss, val_acc, val_unb_acc = test(model, valid_loader, f_loss, device)
    print(" Validation : Loss : {:.4f}, Acc : {:.4f}, Unb.Acc. : {:.4f}".format(val_loss, val_acc, val_unb_acc))

    model_checkpoint.update(val_loss)


model.load_state_dict(torch.load(model_path))

# Switch to eval mode 
model.eval()

test_loss, test_acc, test_unb_acc = test(model, valid_loader, f_loss, device)
print("\n\n Test : Loss : {:.4f}, Acc. : {:.4f}, Unb.Acc. : {:.4f}".format(test_loss, test_acc, test_unb_acc))

In [ ]:
#Plot the confusion matrix
def confusion(model, loader, device):
    """
    Display the confusion matrix after iterating over the loader

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        device    -- The device to use for computation 

    """
    # We disable gradient computation which speeds up the computation
    # and reduces the memory usage
    with torch.no_grad():
        # We enter evaluation mode. This is useless for the linear model
        # but is important with layers such as dropout, batchnorm, ..
        model.eval()
        N = 0
        TP, FP, TN, FN = 0.0, 0.0, 0.0, 0.0
        for i, (inputs, targets) in enumerate(loader):
            # We got a minibatch from the loader within inputs and targets
            
            # We need to copy the data on the GPU if we use one
            inputs, targets = inputs.to(device), targets.to(device)
            N += inputs.shape[0]
                
            # Compute the forward pass, i.e. the scores for each input
            outputs = model(inputs)
            predicted_targets = outputs.argmax(dim=1)
            
            #Compute coefficients
            TP += ((predicted_targets == targets) & (targets == 1)).sum().item()
            FP += ((predicted_targets != targets) & (targets == 0)).sum().item()
            TN += ((predicted_targets == targets) & (targets == 0)).sum().item()
            FN += ((predicted_targets != targets) & (targets == 1)).sum().item()

        cm_list = [['{0:.2%}'.format(TP/N),'{0:.2%}'.format(FN/N)],['{0:.2%}'.format(FP/N),'{0:.2%}'.format(TN/N)]] 
        cm_list[0].insert(0,'Real Rejected')
        cm_list[1].insert(0,'Real Accepted')
        print(tabulate(cm_list,headers=['Real/Pred','Pred Rejected', 'Pred Accepted']))

confusion(model, valid_loader, device)

# Explicability

In [ ]:
#Get features' names
features = list(application.columns)[1:]
nb_features = 5

### General analysis

In [ ]:
#Measure the features impact on decision according to layers' weights
#In the following analysis we ignore the impact of bias and ReLu
layers_array = []

for layer_param in list(model.parameters()) :
    try : 
        a,b = layer_param.size() #Parameters are the weights of inputs
        layers_array.append(np.array(layer_param.data))
        
    except : 
        #These paremeters are related to bias
        continue

In [ ]:
#Look at filters used by the neural network
#Usefull for images analysis or any NN where data is ordered. 
#Add a regularization L1 to see if results are better
#Try to order inputs by topics (job, revenu, status) and add legend

import matplotlib.pyplot as plt

#We compute the relationship with inputs for every neurons by recursive matrix product
layer_weight = np.identity(len(features))
for ind in range(0,len(layers_array)) : 
    layer_weight = np.dot(layers_array[ind], layer_weight)
    plt.imshow(layer_weight)
    plt.title("Layer %s" %(ind+1))
    plt.colorbar()
    plt.show()

In [ ]:
#Print the more determinant features

def invert_normal(value, normal_coefs) : 
    return (value*normal_coefs[1]) #No interest to add the average as we do a difference before calling this functions

def top_features(list_weight, normal=False) :
    if normal :
        return list(map(lambda x : [features[x],"{:.2f}".format(invert_normal(list_weight[x],normal_coefs[x]))], list_weight.argsort()[:nb_features]))
    return list(map(lambda x : [features[x],"{:.2f}".format(list_weight[x])], list_weight.argsort()[:nb_features]))

def last_features(list_weight, normal=False) :
    if normal : 
        return list(map(lambda x : [features[x],"{:.2f}".format(invert_normal(list_weight[x],normal_coefs[x]))], list_weight.argsort()[-nb_features:]))   
    return list(map(lambda x : [features[x],"{:.2f}".format(list_weight[x])], list_weight.argsort()[-nb_features:]))


print('%s more determinant features for loan attribution' % (nb_features*2))
disp_tab = top_features(layer_weight[0])
disp_tab.append(['...','...'])
disp_tab += last_features(layer_weight[0])
print(tabulate(disp_tab[::-1], headers=['Features','weight']))

print('\n\n%s more determinant features for loan rejection' % (nb_features*2))
disp_tab = top_features(layer_weight[1])
disp_tab.append(['...','...'])
disp_tab += last_features(layer_weight[1])
print(tabulate(disp_tab[::-1], headers=['Features','weight']))

### Focusing on specific candidates

In [ ]:
#Select clients
nb_samples = 5
target = 1 #0 loan accepted, 1 refused

def select_samples(model, loader, nb_samples, target):
    """
    Return "nb_samples" estimations of the neural network of the target "target"

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        nb_samples -- Number of samples to get
        target    -- Desired target

    """
    
    # We disable gradient computation which speeds up the computation
    # and reduces the memory usage
    with torch.no_grad():
        # We enter evaluation mode. This is useless for the linear model
        # but is important with layers such as dropout, batchnorm, ..
        model.eval()
        
        #Defining the returned tensors
        inputs_tensor, outputs_tensor = torch.Tensor(), torch.Tensor()
        current_samples = 0
        total_samples = len(loader.dataset)
        
        while current_samples < nb_samples :
            idx = int(np.random.random()*total_samples) #We chose randomly an index
            inpt = valid_loader.dataset[idx][0].reshape(1,-1)
            outpt = model(inpt) #We keep only the inputs, not the target
            if outpt.argmax(dim=1) == target : 
                inputs_tensor = torch.cat([inputs_tensor, inpt])
                outputs_tensor = torch.cat([outputs_tensor, outpt])
                current_samples += 1
        
        return inputs_tensor, outputs_tensor
            
inputs, outputs = select_samples(model, valid_loader, nb_samples, target)

In [ ]:
#Print the confidence across samples
import matplotlib.pyplot as plt
from torch.nn.functional import softmax

confidence = softmax(outputs).T
configs = confidence[0]
N = len(configs)
ind = np.arange(N)

width = 0.4

p1 = plt.bar(ind, confidence[0], width, color='g')
p2 = plt.bar(ind, confidence[1], width, bottom=confidence[0], color='r')

plt.ylim([0,1.2])
plt.ylabel('Probability', fontsize=12)
plt.xlabel('Samples', fontsize=12)
plt.legend((p1[0], p2[0]), ('Accepted', 'Refused'), fontsize=12, ncol=2, framealpha=0, fancybox=True)
plt.show()

In [ ]:
#Take a look at the microvariations for a client to change its class

#Improvment : Stop the gradient descent on incompatible direction / focus only on specific axis (as salary, status, not job...)

model.eval()

for i in range(nb_samples) : 
    shift_input = inputs[i].reshape(1,-1)
    shift_input.requires_grad_(True)
    outpt = model(shift_input)

    while outpt.argmax(dim=1) == target : 
        loss = abs(outpt[0,0]-(target)) + abs(outpt[0,1]-(1-target)) #We look how to modify the inputs to go in the other class
        loss.backward()
        grad = shift_input.grad.detach().clone()
        shift_input.requires_grad_(False)
        shift_input = shift_input.detach().clone() - 0.01*grad
        shift_input.requires_grad_(True)
        outpt = model(shift_input)
    
    shift = shift_input - inputs[i].reshape(1,-1)
    
    
    
    print(f'\nThe status of the Client {i} would have changed if the following {2*nb_features} features were modified')
    disp_tab = top_features(shift.reshape(-1), normal=True)
    disp_tab.append(['...','...'])
    disp_tab += last_features(shift.reshape(-1), normal=True)
    print(tabulate(disp_tab[::-1], headers=['Features','Shift']))

In [ ]:
#Do a KNN on the client properties (more adapted to this specific problem)